In [1]:
import numpy as np

#math & plot
import matplotlib.pyplot as plt
%matplotlib inline

#sklearn
import sklearn.metrics as metrics
from sklearn.externals import joblib

#debug purposes
%load_ext autoreload
%autoreload 2

In [2]:
#parameters
target_n_trees = 100
global_n_jobs = 8

# Extracting the trained model

In [3]:
import cPickle
def cDump(obj,fname):
    with open(fname,'w') as f:
        cPickle.dump(obj,f)
def cLoad(fname):
    with open(fname,'r') as f:
        return cPickle.load(f)
        
from StringIO import StringIO
import _matrixnetapplier as mnet

In [4]:
with open('../formula/MSLR10k_ef.mx', 'r') as f:
    formula = mnet.MatrixnetClassifier(StringIO(cPickle.load(f))) #btw he's a regressor, not classifier

depth, nTrees, itr = formula.iterate_trees().next()
trees = [tree for tree in itr]
print len(trees), '==',nTrees

9997 == 9997


# Loading dataset

In [5]:
import io_ranking as io

Failed to import h5py. H5 IO operations will be unavailable


In [6]:
%%time
##warning! this can take a long time. no need to rerun that code if u have CSV files created once.
#io.save_csv("../data/MSLR10/Fold1/train.txt","../data/MSLR10/mslr_train")
#io.save_csv("../data/MSLR10/Fold1/test.txt","../data/MSLR10/mslr_test")
#io.save_csv("../data/MSLR10/Fold1/vali.txt","../data/MSLR10/mslr_vali")
print "converted that"

converted that
CPU times: user 30 µs, sys: 14 µs, total: 44 µs
Wall time: 42.2 µs


In [7]:
#load training set
Xtr,Qtr,Ytr = io.load_csv("../data/MSLR10/mslr_train")
Xts,Qts,Yts = io.load_csv("../data/MSLR10/mslr_test")

reading from ../data/MSLR10/mslr_train
done
reading from ../data/MSLR10/mslr_test
done


In [8]:
from factory import RegressionFactory
#DataFactory is just a data wrapper that can handle splits, predictions, etc. 
#Used to avoid recomputing metadata at each predictions and passing large argument strings
trainFactory = RegressionFactory(Xtr,Ytr)
testFactory = RegressionFactory(Xts,Yts)

In [9]:
del Xtr,Ytr,Xts,Yts

In [10]:
trainFactory.labels = trainFactory.labels.astype('int8')
trainFactory.events = trainFactory.events.astype('float32')
trainFactory.weights = trainFactory.weights.astype('int8')

In [11]:
print "train: ",trainFactory.events.shape,"qids:",len(set(Qtr))
print "test: ",testFactory.events.shape,"qids:",len(set(Qts))
print "qid intersection:",len(set.intersection(set(Qtr),set(Qts))),"(must be 0)"

train:  (723412, 136) qids: 6000
test:  (241521, 136) qids: 2000
qid intersection: 0 (must be 0)


# greedy pruning for the whole data

In [12]:
import greedy
from loss_functions import MSELoss

In [13]:
%%time
read = True
fname="../dumps/last_greedy.pcl"
if not read:
    trees_greedy = greedy.greed_up_features_bfs(trees,trainFactory,
                                              loss = MSELoss,
                                              learning_rate = .35,
                                              learning_rate_decay=1.,# no decay
                                              nTrees = target_n_trees,
                                              trees_sample_size =500, #chosen from the ensemble at random each iteration
                                              verbose = True,
                                              regularizer=0.0005*len(trainFactory.labels), #added to gradient walker's leaf denominator
                                              use_joblib=True,
                                              n_jobs=global_n_jobs, #threading by default
                                              )
    cDump(trees_greedy,fname)
else:
    trees_greedy = cLoad(fname)
    


iteration # 0  ntrees =  1 
best loss =  463018.540185
learning_rate =  0.35
sample_size 500

iteration # 1  ntrees =  2 
best loss =  444651.715193 
last loss =  444651.715193
learning_rate =  0.35
sample_size 500

iteration # 2  ntrees =  3 
best loss =  434686.18885 
last loss =  434686.18885
learning_rate =  0.35
sample_size 500

iteration # 3  ntrees =  4 
best loss =  428073.844285 
last loss =  428073.844285
learning_rate =  0.35
sample_size 500

iteration # 4  ntrees =  5 
best loss =  423237.158698 
last loss =  423237.158698
learning_rate =  0.35
sample_size 500

iteration # 5  ntrees =  6 
best loss =  420240.959481 
last loss =  420240.959481
learning_rate =  0.35
sample_size 500

iteration # 6  ntrees =  7 
best loss =  417492.989411 
last loss =  417492.989411
learning_rate =  0.35
sample_size 500

iteration # 7  ntrees =  8 
best loss =  415322.558968 
last loss =  415322.558968
learning_rate =  0.35
sample_size 500

iteration # 8  ntrees =  9 
best loss =  413494.09916

In [14]:
from greedy import PrunedFormula as pf
bias = 0.0# NOT np.average(trainFactory.labels)
trees_stupid = pf(trees[:target_n_trees],bias)
trees_full = pf(trees,bias)

y_pred_stupid = trees_stupid.predict(testFactory)
y_pred_full = trees_full.predict(testFactory)

In [15]:
y_pred_greedy = trees_greedy.predict(testFactory)

In [16]:
print metrics.mean_squared_error(testFactory.labels,y_pred_greedy),
print metrics.mean_squared_error(testFactory.labels,y_pred_stupid),
print metrics.mean_squared_error(testFactory.labels,y_pred_full)
print "well..."

0.565785832991 0.695731252371 0.554568902063
well...


In [17]:
print len(trainFactory.labels)

723412


In [18]:
read = True
fname = "../dumps/last_grid.pcl"
if not read:
    trees_grid = {}
    for lr in [0.35,0.4,0.5]:
            for r in [0,0.001]:
                _formula = greedy.greed_up_features_bfs(trees,trainFactory,
                                                      loss = MSELoss,
                                                      learning_rate = lr,
                                                      learning_rate_decay=1.,# no decay
                                                      nTrees = target_n_trees,
                                                      trees_sample_size =500, #chosen from the ensemble at random each iteration
                                                      verbose = True,
                                                      regularizer=r*len(trainFactory.labels), #added to gradient walker's leaf denominator
                                                      use_joblib=True,
                                                      n_jobs=global_n_jobs, #threading by default
                                                      )
                trees_grid[(lr,r)] = _formula
                
    cDump(trees_grid, fname)
else:
    trees_grid = cLoad(fname)



iteration # 0  ntrees =  1 
best loss =  459613.857508
learning_rate =  0.35
sample_size 500

iteration # 1  ntrees =  2 
best loss =  442190.186901 
last loss =  442190.186901
learning_rate =  0.35
sample_size 500

iteration # 2  ntrees =  3 
best loss =  432770.493453 
last loss =  432770.493453
learning_rate =  0.35
sample_size 500

iteration # 3  ntrees =  4 
best loss =  426551.32343 
last loss =  426551.32343
learning_rate =  0.35
sample_size 500

iteration # 4  ntrees =  5 
best loss =  421438.664031 
last loss =  421438.664031
learning_rate =  0.35
sample_size 500

iteration # 5  ntrees =  6 
best loss =  418032.44535 
last loss =  418032.44535
learning_rate =  0.35
sample_size 500

iteration # 6  ntrees =  7 
best loss =  415542.623874 
last loss =  415542.623874
learning_rate =  0.35
sample_size 500

iteration # 7  ntrees =  8 
best loss =  413525.319562 
last loss =  413525.319562
learning_rate =  0.35
sample_size 500

iteration # 8  ntrees =  9 
best loss =  411799.900479 

In [19]:
res = []
for key in trees_grid:
    _formula = trees_grid[key]
    y_pred_sanity = _formula.predict(testFactory)
    print key, metrics.mean_squared_error(testFactory.labels,y_pred_sanity)
    res.append((key, metrics.mean_squared_error(testFactory.labels,y_pred_sanity)))

(0.35, 0.001) 0.565079919559
(0.5, 0.001) 0.566144651233
(0.4, 0.001) 0.563709997282
(0.4, 0) 0.565228472769
(0.35, 0) 0.564526227171
(0.5, 0) 0.567501400434


#old hierarchy

In [ ]:
#usability distribution
read = True
fname = '../dumps/thresholds.pcl'
if not read:
    thresholds = mnet.get_thresholds(trees,formula.feature_ids,0.001)
    #todo: make a LOGICALLY CONSISTENT parallel threshold extractor AND criteria selector. Right now joblib slows things down
    cDump(thresholds,fname)
else:    
    thresholds = cLoad(fname)

In [ ]:
plt.scatter(range(len(thresholds)),thresholds[:,2])
print sum(thresholds[:,2] >150)

## Criteria selection

In [ ]:
import hierarchy as old_hierarchy

In [ ]:
#usability distribution
read = False
fname = '../dumps/criteria.pcl'
if not read:
    thresholds_active = thresholds[thresholds[:,2]>100] #at least 100 times used in the original ensemble
    print len(thresholds_active)
    criteria = hierarchy.select_criteria(trainFactory,thresholds_active,4,True)
    cDump(criteria,fname)
else:
    criteria = cLoad(fname)

In [ ]:
criteria

In [ ]:
#split = hierarchy.split_upper(trainFactory,criteria,equalizeWeights=False,split_weights=1.,split_inclusion=.7) 
#при каждом разделении в подвыборку  попадает split_inclusion примеров из другой половины выборки с весом split_weights
#print [split[i].events.shape[0] for i in split]
#print [sum(split[i].weights) for i in split]

##models

In [ ]:
%%time
read = False
fname = "../dumps/last_old_hierarchy.pcl"
if not read:
    
    trees_splitted = hierarchy.train_splitted_boosts(trees, trainFactory,criteria,
                                                     breadth = 1,
                                                     loss = MSELoss,
                                                     learning_rate = 0.25, 
                                                     nTrees_leaf= target_n_trees,
                                                     trees_sample_size=500,
                                                     regularizer =0.0001,
                                                     verbose=True,
                                                     use_joblib = True,n_jobs = global_n_jobs,
                                                     joblib_backend = "multiprocessing",
                                                     weights_outside_leaf = 0.75, inclusion_outside_leaf = 0.75) 
    cDump(trees_splitted,fname)
else:
    trees_splitted = cLoad(fname)

In [ ]:
y_pred_splitted= trees_splitted.predict(testFactory)

In [ ]:
w_test = testFactory.weights
Yts = testFactory.labels
print 'spltd\t',metrics.mean_squared_error(Yts,y_pred_splitted)
print 'greedy\t',metrics.mean_squared_error(Yts,y_pred_greedy)
print 'stupid\t',metrics.mean_squared_error(Yts,y_pred_stupid)
print 'full\t',metrics.mean_squared_error(Yts,y_pred_full)
print "well..."

#MSE learning curves

In [ ]:
def learning_curve(formula,factory, metric,n_points = None):
    
    lcurve = []

    Ypred = np.zeros(len(factory.labels))
                  
    for i,tree_pred in enumerate(formula.staged_predict(factory)):

        Ypred += tree_pred
        lcurve.append(metric(factory.labels, Ypred,sample_weight = factory.weights))
        if n_points is not None and i >= n_points:
            break
    while n_points is not None and i < n_points:
        i+=1
        lcurve.append(lcurve[-1])
        
    return lcurve

In [ ]:
metric_name = 'MSE'
n_trees = target_n_trees
metric = metrics.mean_squared_error

stupid_lcurve = learning_curve(trees_stupid,testFactory,metric,n_trees)
greedy_lcurve = learning_curve(trees_greedy,testFactory,metric,n_trees)
splitted_lcurve = learning_curve(trees_splitted,testFactory,metric,n_trees)

full_line = metric(testFactory.labels,y_pred_full)

p = range(n_trees+1)

plt.figure(figsize = [14,14])
plt.plot(p,[full_line for i in p],label = "full")
plt.plot(p,stupid_lcurve,label = "stupid")
plt.plot(p,greedy_lcurve,label = "greedy")
plt.plot(p,splitted_lcurve,label = "splitted")
plt.title('learning curves('+metric_name+')')
plt.legend(loc="lower right")
